In [2]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('dataframeAPI').getOrCreate()

In [3]:
training = spark.read.csv('test4.csv',header=True,inferSchema=True)

In [4]:
training.show()

+-----+---+----------+------+
|Name |Age|Experience|Salary|
+-----+---+----------+------+
|Aryan| 20|        34| 69000|
|Grace| 19|        29|220000|
|Sneha| 19|        78|170000|
| Rene| 33|        22| 22000|
| John| 20|        11| 18000|
| Isen| 22|        45| 90000|
| Arlo| 19|        12|120000|
|Blyke| 21|        22|200000|
+-----+---+----------+------+



In [7]:
training.columns

['Name ', 'Age', 'Experience', 'Salary']

In [8]:
[Age,Experience]---->new feature--->independent feature

SyntaxError: invalid syntax (1836357362.py, line 1)

In [9]:
from pyspark.ml.feature import VectorAssembler
featureasembler = VectorAssembler(inputCols=['Age','Experience'],outputCol='Independent Feature')

In [11]:
output = featureasembler.transform(training)

In [12]:
output.show()

+-----+---+----------+------+-------------------+
|Name |Age|Experience|Salary|Independent Feature|
+-----+---+----------+------+-------------------+
|Aryan| 20|        34| 69000|        [20.0,34.0]|
|Grace| 19|        29|220000|        [19.0,29.0]|
|Sneha| 19|        78|170000|        [19.0,78.0]|
| Rene| 33|        22| 22000|        [33.0,22.0]|
| John| 20|        11| 18000|        [20.0,11.0]|
| Isen| 22|        45| 90000|        [22.0,45.0]|
| Arlo| 19|        12|120000|        [19.0,12.0]|
|Blyke| 21|        22|200000|        [21.0,22.0]|
+-----+---+----------+------+-------------------+



In [14]:
final_data=output.select('Independent Feature','Salary')
final_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [20.0,34.0]| 69000|
|        [19.0,29.0]|220000|
|        [19.0,78.0]|170000|
|        [33.0,22.0]| 22000|
|        [20.0,11.0]| 18000|
|        [22.0,45.0]| 90000|
|        [19.0,12.0]|120000|
|        [21.0,22.0]|200000|
+-------------------+------+



In [17]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=final_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Feature',labelCol='Salary')
regressor=regressor.fit(train_data)

In [18]:
regressor.coefficients

DenseVector([-2196.4398, 1057.4819])

In [20]:
regressor.intercept

143248.0437604223

In [21]:
pred_res=regressor.evaluate(test_data)

In [23]:
pred_res.predictions.show()

+-------------------+------+-----------------+
|Independent Feature|Salary|       prediction|
+-------------------+------+-----------------+
|        [22.0,45.0]| 90000|142513.0539588365|
|        [33.0,22.0]| 22000| 94030.1316521469|
+-------------------+------+-----------------+



In [24]:
pred_res.meanAbsoluteError,pred_res.meanSquaredError

(62271.5928054917, 3972980350.9546447)